# Kafkaの起動を行います

/home/pyspark/kafka/bin/kafka-topics.sh \
    --create --topic pyspark-topic1 \
    --replication-factor 1 \
    --partitions 3 \
    --bootstrap-server kafka:9092 

In [1]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [2]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。

:: loading settings :: url = jar:file:/home/pyspark/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pyspark/.ivy2/cache
The jars for the packages stored in: /home/pyspark/.ivy2/jars
org.apache.spark#spark-streaming_2.13 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c84f86a5-096c-4d2c-9c45-abe0cdec7319;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in c

# WebUIでライブラリが保存されているか確認してみましょう

100000

In [3]:
# PySparkをKafkaと接続します

# Subscribe to 1 topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "pyspark-topic1") \
  .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").show()

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
kafka

In [7]:
ds = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("topic", "pyspark-topic1") \
  .option("checkpointLocation","/tmp/kafka") \
  .start()
ds.awaitTermination()

21/11/30 12:13:39 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
21/11/30 12:13:39 WARN StreamingQueryManager: Stopping existing streaming query [id=7aa56c59-0860-455e-89c4-0488457a952f, runId=bda4094e-41d5-43e7-ad27-ee5795de28af], as a new run is being started.


In [ ]:
# kafkaのダウンロード
wget https://dlcdn.apache.org/kafka/3.0.0/kafka-3.0.0-src.tgz

In [ ]:
bin/kafka-topics.sh \
    --delete --topic quickstart-events \
    --bootstrap-server localhost:9092 

In [ ]:
# 繋ぐコマンド
# ちょこちょこバグがあって、接続できない時があるので、今回の講座で使っているバージョンをおさらいします。

# Spark 3.2
# Java11(openjdk version "11.0.12)
# Kafkka(3.0.0)
# 
pyspark --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.3

In [1]:
spark.stop()
spark.sparkContext.stop()

NameError: name 'spark' is not defined